In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [3]:
from tensorflow import keras
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [4]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [5]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [6]:
keras_reg.fit(X_train, y_train, epochs=10, verbose=1, 
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/10
363/363 [==============================] - 0s 1ms/step - loss: 1.0896 - val_loss: 20.7721
Epoch 2/10
363/363 [==============================] - 0s 837us/step - loss: 0.7606 - val_loss: 5.0266
Epoch 3/10
363/363 [==============================] - 0s 804us/step - loss: 0.5456 - val_loss: 0.5490
Epoch 4/10
363/363 [==============================] - 0s 804us/step - loss: 0.4732 - val_loss: 0.4529
Epoch 5/10
363/363 [==============================] - 0s 785us/step - loss: 0.4503 - val_loss: 0.4188
Epoch 6/10
363/363 [==============================] - 0s 807us/step - loss: 0.4338 - val_loss: 0.4129
Epoch 7/10
363/363 [==============================] - 0s 791us/step - loss: 0.4241 - val_loss: 0.4004
Epoch 8/10
363/363 [==============================] - 0s 835us/step - loss: 0.4168 - val_loss: 0.3944
Epoch 9/10
363/363 [==============================] - 0s 793us/step - loss: 0.4108 - val_loss: 0.3961
Epoch 10/10
363/363 [==============================] - 0s 815us/step - loss: 0.4060

In [9]:
mse_test = keras_reg.score(X_test, y_test)
mse_test

162/162 [==============================] - 0s 519us/step - loss: 0.3998


-0.3998211622238159

In [8]:
y_pred = keras_reg.predict(X_test)

In [9]:
np.random.seed(42)
tf.random.set_seed(42)

In [10]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=5,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] learning_rate=0.001683454924600351, n_hidden=0, n_neurons=15 ....
Epoch 1/5


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


242/242 [==============================] - 0s 1ms/step - loss: 3.3071 - val_loss: 1.3735
Epoch 2/5
242/242 [==============================] - 0s 884us/step - loss: 0.9782 - val_loss: 0.6881
Epoch 3/5
242/242 [==============================] - 0s 839us/step - loss: 0.6186 - val_loss: 0.5887
Epoch 4/5
242/242 [==============================] - 0s 843us/step - loss: 0.5504 - val_loss: 0.5466
Epoch 5/5
121/121 [==============================] - 0s 1ms/step - loss: 0.5399
[CV]  learning_rate=0.001683454924600351, n_hidden=0, n_neurons=15, total=   1.4s
[CV] learning_rate=0.001683454924600351, n_hidden=0, n_neurons=15 ....
Epoch 1/5
  1/242 [..............................] - ETA: 0s - loss: 5.5857

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s


242/242 [==============================] - 0s 1ms/step - loss: 3.1141 - val_loss: 4.8527
Epoch 2/5
242/242 [==============================] - 0s 843us/step - loss: 1.1858 - val_loss: 5.7781
Epoch 3/5
242/242 [==============================] - 0s 839us/step - loss: 0.7680 - val_loss: 7.3954
Epoch 4/5
242/242 [==============================] - 0s 847us/step - loss: 0.6629 - val_loss: 9.0480
Epoch 5/5
121/121 [==============================] - 0s 570us/step - loss: 0.8604
[CV]  learning_rate=0.001683454924600351, n_hidden=0, n_neurons=15, total=   1.4s
[CV] learning_rate=0.001683454924600351, n_hidden=0, n_neurons=15 ....
Epoch 1/5
242/242 [==============================] - 0s 1ms/step - loss: 3.3292 - val_loss: 1.4882
Epoch 2/5
242/242 [==============================] - 0s 938us/step - loss: 1.0465 - val_loss: 0.7937
Epoch 3/5
242/242 [==============================] - 0s 843us/step - loss: 0.6751 - val_loss: 0.6177
Epoch 4/5
242/242 [==============================] - 0s 868us/step - los

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   44.9s finished


RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x0000022BE57FEE10>, as the constructor either does not set or modifies parameter learning_rate

In [11]:
rnd_search_cv.best_params_

{'learning_rate': 0.008339092654580042, 'n_hidden': 1, 'n_neurons': 38}

In [12]:
rnd_search_cv.best_score_

-0.4187048574288686

In [13]:
rnd_search_cv.best_estimator_

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_'

In [14]:
rnd_search_cv.score(X_test, y_test)

AttributeError: 'RandomizedSearchCV' object has no attribute 'scorer_'

In [15]:
model = rnd_search_cv.best_estimator_.model
model

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_'

In [16]:
model.evaluate(X_test, y_test)

NameError: name 'model' is not defined